In [1]:
from spacy.tokens import DocBin
import spacy
import json
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [2]:
from spacy.util import filter_spans
nlp = spacy.blank("en") 
def create_training(TRAIN_DATA):
    db = DocBin() # create a DocBin object
    for text, annot in tqdm(TRAIN_DATA): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
            if span is None:
                pass
            else:
                ents.append(span)
            
        doc.ents = filter_spans(ents) # remove duplicates
        db.add(doc)
    return (db)

In [4]:
with open ("data/train_data.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open ("data/valid_data.json", "r", encoding="utf-8") as f:
    valid_data = json.load(f)

In [5]:
print(train_data[0])

["Bộ Công Thương vừa có 'động thái mạnh mẽ, đầu tiên trong các bộ'", {'entities': [[0, 14, 'ORGANIZATION']]}]


### From json -> spacy files

In [6]:
vlsp_2018_train = create_training(train_data)
vlsp_2018_train.to_disk("data/vlsp_2018_train.spacy")

vlsp_2018_valid = create_training(valid_data)
vlsp_2018_valid.to_disk("data/vlsp_2018_valid.spacy")

100%|██████████| 2209/2209 [00:01<00:00, 1379.13it/s]


### Download base_config.cfg and make configuration from https://spacy.io/usage/training, then convert to the final training format

In [7]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


### Being trained on local machine without GPU so it takes time!

In [8]:
!python -m spacy train config.cfg --output ./output --paths.train data/vlsp_2018_train.spacy --paths.dev data/vlsp_2018_valid.spacy

ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-11-04 19:59:55,700] [INFO] Set up nlp object from config
[2021-11-04 19:59:55,723] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-11-04 19:59:55,730] [INFO] Created vocabulary
[2021-11-04 19:59:55,730] [INFO] Finished initializing nlp object
[2021-11-04 20:00:00,042] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     47.00    0.00    0.00    0.00    0.00
  0     200       1078.87   2622.82   39.27   50.34   32.19    0.39
  0     400       5218.47   1767.10   57.30   60.80   54.18    0.57
  0     600        282.87   1483.90   67.95   69.63   66.35    0.68
  0     800 

In [3]:
with open ("data/test_data.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

vlsp_2018_test = create_training(test_data)
vlsp_2018_test.to_disk("data/vlsp_2018_test.spacy")

100%|██████████| 2611/2611 [00:02<00:00, 892.34it/s] 


### Evalutation time!!!

In [4]:
!python -m spacy evaluate output/model-best data/vlsp_2018_test.spacy

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   60.06 
NER R   53.14 
NER F   56.39 
SPEED   22665 


=============================== NER (per type) ===============================

                    P       R       F
PERSON          67.87   57.19   62.07
ORGANIZATION    39.67   33.68   36.43
LOCATION        65.36   67.81   66.56
MISCELLANEOUS   38.78   12.79   19.24



In [9]:
nlp = spacy.load("output/model-best")

doc = nlp("Trận Việt Nam - Campuchia được chuyển ra Mỹ Đình thay vì Thống Nhất, \
 sân Mỹ Đình sẽ là địa điểm tổ chức trận đấu giữa đội tuyển \
Việt Nam và đội tuyển Campuchia vào ngày 10.10 tới đây.")

for ent in doc.ents:
    print(ent, "----", ent.label_)

Việt Nam ---- LOCATION
Campuchia ---- LOCATION
Mỹ Đình ---- LOCATION
Thống Nhất ---- LOCATION
sân Mỹ Đình ---- LOCATION
Việt Nam ---- LOCATION
Campuchia ---- LOCATION


### The results are pretty good!

In [10]:
doc = nlp("Giống với mọi điệp viên khác ông sử dụng rất nhiều cái tên \
như Pierre Vũ Ngọc Nhạ, Vũ Ngọc Nhã, Hoàng Đức Nhã, Hai Long, \
…nhưng người ta biết tới Thiếu tướng Vũ Ngọc Nhạ nhiều nhất vẫn \
qua biệt danh 'Ông cố vấn'. Vũ Ngọc Nhạ sinh năm 1928 tại Thái Bình, từ nhỏ ông \
đã sống tại quê mẹ ở Giáo xứ Phát Diệm, Ninh Bình.")

for ent in doc.ents:
    print(ent, "----", ent.label_)

Pierre Vũ Ngọc Nhạ ---- PERSON
Vũ Ngọc Nhã ---- PERSON
Hoàng Đức Nhã ---- PERSON
Hai Long ---- LOCATION
Vũ Ngọc Nhạ ---- PERSON
Vũ Ngọc Nhạ ---- PERSON
Thái Bình ---- LOCATION
Phát Diệm ---- LOCATION
Ninh Bình ---- LOCATION


### The custom model did well in recognizing entities, however sometime mislabelling
-> We can tackle this by making it **compound words** -> spacy.blank('vi')

In [ ]:
#!pip install pyvi
#!pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz

In [17]:
vietnamese = spacy.blank('vi')
doc = vietnamese("Giống với mọi điệp viên khác ông sử dụng rất nhiều cái tên \
như Pierre Vũ Ngọc Nhạ, Vũ Ngọc Nhã, Hoàng Đức Nhã, Hai Long, \
…nhưng người ta biết tới Thiếu tướng Vũ Ngọc Nhạ nhiều nhất vẫn \
qua biệt danh 'Ông cố vấn'. Vũ Ngọc Nhạ sinh năm 1928 tại Thái Bình, từ nhỏ ông \
đã sống tại quê mẹ ở Giáo xứ Phát Diệm, Ninh Bình.")
print(doc)


Giống với mọi điệp_viên khác ông sử_dụng rất nhiều cái tên như Pierre_Vũ_Ngọc_Nhạ, Vũ_Ngọc_Nhã, Hoàng_Đức_Nhã, Hai Long, …nhưng người_ta biết tới Thiếu_tướng Vũ_Ngọc_Nhạ nhiều nhất vẫn qua biệt_danh 'Ông cố_vấn'.Vũ_Ngọc_Nhạsinhnăm1928tạiThái_Bình, từnhỏ ôngđãsốngtạiquêmẹở Giáo_xứ Phát_Diệm ,Ninh_Bình.


### However, vietnamese language in spacy at this time is not good enough at capturing compound words, leading to another problem to solve.
### For the sake of simplicity, the types of entities in the dataset are easy to label as long as entities are found.